In [ ]:
import os

data_loc = "/data/"

os.environ["OPENBLAS_NUM_THREADS"] = "16"
os.environ["NUM_THREADS"] = "16"
os.environ["OMP_NUM_THREADS"] = "16"
import numpy as np
import tensorflow as tf
from functions import process_label_maps, generate_patches
from tensorflow.keras.models import load_model
import matplotlib
import numpy as np
import xarray as xr
matplotlib.rcParams.update({'font.size': 16})
from scipy import ndimage
import joblib
import glob
from scipy.spatial import cKDTree
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)




2025-01-08 14:52:19.888327: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21033 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


1 Physical GPUs, 1 Logical GPUs


In [ ]:
bands=[31]
modis_folder = "loc_of_MODIS_npz_files"

In [ ]:
from scipy.ndimage import zoom
from extract_modis_data import process_npy_file
max_zenith=47.4
zenith = np.load("%sland_sea_ice_mask/sensor_zenith_bilinear_1km.npy" %data_loc)
zenith_mask = zenith < max_zenith
ds_water_mask=xr.open_dataset(f"{data_loc}/land_sea_ice_mask/nimbus/with_lonlat/NSIDC0051_SEAICE_PS_N25km_20200302_v2.0.nc")

water_mask_ravel = None
coords_lowres = np.vstack((ds_water_mask.lat.values.ravel(), ds_water_mask.lon.values.ravel())).T
tree = cKDTree(coords_lowres)
from autoencoder import SimpleAutoencoder
bands = [31]
band_str = ["_" + str(b) for b in bands]
band_str = "".join(band_str) 
# model_run_name = f"dnb_ice01_l95_z50_ps128_band{band_str}_6filters"

band_str = "31"
model_run_name = f"dnb_ice01_l95_z50_ps128_band{band_str}"
patch_load_name = f"dnb_ice01_l95_z50_ps128_band{band_str}"

max_vals = np.load(f"{data_loc}models/patch_size128/filter128/max_val_{patch_load_name}_2019-2023.npy")
min_vals = np.load(f"{data_loc}models/patch_size128/filter128/min_val_{patch_load_name}_2019-2023.npy")
autoencoder_predict = SimpleAutoencoder(len(bands), 128, 128)
encoder = load_model(f"{data_loc}models/patch_size128/filter128/encoder_{model_run_name}.h5")

strides = 128

total cores: 256


In [ ]:

years_period = ["2000", "2001", "2002", "2003", "2004", "2005"]
modis_start_date = 55

def get_large_caos_for_years(years_period, modis_start_date):
    dates_large_cao = []
    times_large_cao = []

    for year in years_period:
        ds = np.load(f"/uio/kant/geo-geofag-u1/fslippe/data/models/patch_size128/filter128/clustering/K14/cao_date_time_lists/times_dnb_ice01_l95_z50_ps128_band31_nK14_thr30_{year}0101-{year}0531.npy", allow_pickle=True).item()
        dates_large_cao.append(ds["dates"])
        times_large_cao.append(ds["times"])
        if year != "2024":
            ds = np.load(f"/uio/kant/geo-geofag-u1/fslippe/data/models/patch_size128/filter128/clustering/K14/cao_date_time_lists/times_dnb_ice01_l95_z50_ps128_band31_nK14_thr30_{year}0901-{year}1231.npy", allow_pickle=True).item()
            dates_large_cao.append(ds["dates"])
            times_large_cao.append(ds["times"])
        if year == years_period[0]:
            for d, t in zip(ds["dates"], ds["times"]):
                if int(d[4:])>=modis_start_date:
                    dates_large_cao.append([d])
                    times_large_cao.append([t])
        elif year == years_period[-1]:
            for d, t in zip(ds["dates"], ds["times"]):
                if int(d[4:])<modis_start_date:
                    dates_large_cao.append([d])
                    times_large_cao.append([t])

    dates_large_cao = np.concatenate(dates_large_cao)
    times_large_cao = np.concatenate(times_large_cao)
    rand_idx = np.random.choice(len(dates_large_cao), 100, replace=False)
    random_choice_dates = dates_large_cao[rand_idx]
    random_choice_times = times_large_cao[rand_idx]

    import glob

    date_list = []
    time_list = []
    x_list = []
    i= 0
    while len(x_list) < 50:
        print("try:", i)
        # Construct the file search pattern
        pattern = f"{modis_folder}{random_choice_dates[i][:4]}/MOD021KM.A{random_choice_dates[i]}.{str(random_choice_times[i]).zfill(4)}*band_31.npz"
        matching_files = glob.glob(pattern)
        result, mask, lon_lat, valid_cols_lon  = process_npy_file(matching_files[0],
                    file_layers=None,
                    bands=[31],
                    max_zenith=47.4,
                    data_loc=data_loc,
                    full_water_mask=None,
                    tree=tree,
                    return_lon_lat=True,
                    npz_compressed=True,
                    lon_min=-45,
                    lon_max=65,
                    lat_min=55,
                    lat_max=82)
        mask = np.where(mask>0.1, 0, 1)

        patches, all_lon_patches, all_lat_patches, starts, ends, shapes, n_patches_tot, indices = generate_patches([result],
                                                                                                                    [mask],
                                                                                                                    [lon_lat],
                                                                                                                    max_vals,
                                                                                                                    min_vals,
                                                                                                                    autoencoder_predict,
                                                                                                                    strides=[1, strides, strides,1])
        with tf.device('/CPU:0'):   
            encoded_patches_cao = encoder.predict(patches)
            encoded_patches_flat_cao = encoded_patches_cao.reshape(encoded_patches_cao.shape[0], -1)
        n_K = 14
        cluster = joblib.load(f"{data_loc}models/patch_size128/filter128/clustering/K{n_K}/cluster_{model_run_name}_filter128_K{n_K}.pkl" )
        labels = cluster.predict(encoded_patches_flat_cao)
        label_map, lon_map, lat_map = process_label_maps(labels,
                                                        all_lon_patches,
                                                        all_lat_patches,
                                                        starts,
                                                        ends,
                                                        shapes,
                                                        indices,
                                                        n_K,
                                                        n_patches_tot,
                                                        128,
                                                        strides,
                                                        [2, 10], 
                                                        [], 
                                                        size_thr_1=None, 
                                                        size_thr_2=None)
        binary_map = np.isin(label_map[0], [6, 9, 10, 2])
        # import:
        labeled_map, num_features = ndimage.label(binary_map)
        region_sizes = ndimage.sum(
            binary_map, labeled_map, range(num_features + 1))

        still_30 = False
        # Loop through each region and check if the region size is below the thr
        # Skipping background (label 0)
        print(result.shape)
        if (random_choice_dates[i], random_choice_times[i]) not in zip(np.array(date_list), np.array(time_list)):
            for region_label in range(1, num_features + 1):
                if region_sizes[region_label] >= 30:
                    still_30 = True
                    break
        if still_30:
            x_list.append(result)
            date_list.append(random_choice_dates[i])
            time_list.append(random_choice_times[i])

        i+=1
    return x_list, date_list, time_list

In [72]:
rand_idx = np.random.choice(300, 60, replace=False)
print(len(np.unique(rand_idx)))

60


In [100]:
years_period = ["2000", "2001", "2002", "2003", "2004", "2005"]
period1, dates1, times1 = get_large_caos_for_years(years_period, modis_start_date)
years_period = ["2005", "2006", "2007", "2008", "2009", "2010"]
period2, dates2, times2 = get_large_caos_for_years(years_period, modis_start_date)
years_period = ["2010", "2011", "2012", "2013", "2014", "2015"]
period3, dates3, times3 = get_large_caos_for_years(years_period, modis_start_date)
years_period = ["2015", "2016", "2017", "2018", "2019", "2020"]
period4, dates4, times4 = get_large_caos_for_years(years_period, modis_start_date)
years_period = ["2020", "2021", "2022", "2023", "2024"]
period5, dates5, times5 = get_large_caos_for_years(years_period, modis_start_date)

try: 0
(3366, 1024, 1)
try: 1
(3697, 1024, 1)
try: 2
(3122, 1024, 1)
try: 3
(3717, 1024, 1)
try: 4
(3376, 1024, 1)
try: 5
(3672, 1024, 1)
try: 6
(4763, 1024, 1)
try: 7
(3897, 1024, 1)
try: 8
(3396, 1024, 1)
try: 9
(3267, 1024, 1)
try: 10
(3544, 1024, 1)
try: 11
(3607, 1024, 1)
try: 12
(4122, 1024, 1)
try: 13
(3793, 1024, 1)
try: 14
(3908, 1024, 1)
try: 15
(3582, 1024, 1)
try: 16
(3507, 1024, 1)
try: 17
(3507, 1024, 1)
try: 18
(3103, 1024, 1)
try: 19
(3118, 1024, 1)
try: 20
(3647, 1024, 1)
try: 21
(3832, 1024, 1)
try: 22
(3724, 1024, 1)
try: 23
(3672, 1024, 1)
try: 24
(4173, 1024, 1)
try: 25
(3222, 1024, 1)
try: 26
(3463, 1024, 1)
try: 27
(3723, 1024, 1)
try: 28
(3486, 1024, 1)
try: 29
(3582, 1024, 1)
try: 30
(4213, 1024, 1)
try: 31
(4113, 1024, 1)
try: 32
(3472, 1024, 1)
try: 33
(3376, 1024, 1)
try: 34
(4182, 1024, 1)
try: 35
(4708, 1024, 1)
try: 36
(3441, 1024, 1)
try: 37
(3486, 1024, 1)
try: 38
(3698, 1024, 1)
try: 39
(3157, 1024, 1)
try: 40
(3548, 1024, 1)
try: 41
(3256, 1024, 1)
tr

In [ ]:
from PIL import Image
def save_images(data, dates, times, max_val, min_val, file_end):
    for i in range(len(data)):
        img_data = np.array((data[i][:,:,0] - min_val) / (max_val - min_val) * 255, dtype=np.uint8)
        img_data = 255 - img_data
        image = Image.fromarray(img_data)
        if os.path.exists(f"/uio/kant/geo-geofag-u1/fslippe/data/figures_for_labeling/full_modis_record/MOD021KM.A{dates[i]}.{str(times[i]).zfill(4)}_{file_end}.jpg"):
            print(f"/uio/kant/geo-geofag-u1/fslippe/data/figures_for_labeling/full_modis_record/MOD021KM.A{dates[i]}.{str(times[i]).zfill(4)}_{file_end}.jpg")
        # print(i, os.path.exists(f"/uio/kant/geo-geofag-u1/fslippe/data/figures_for_labeling/full_modis_record/MOD021KM.A{dates[i]}.{str(times[i]).zfill(4)}_{file_end}.jpg") )
        image.save(f"/uio/kant/geo-geofag-u1/fslippe/data/figures_for_labeling/full_modis_record/MOD021KM.A{dates[i]}.{str(times[i]).zfill(4)}_{file_end}.jpg" )
save_images(period1, dates1, times1, 8, 1, "caos_00-05")    
save_images(period2, dates2, times2, 8, 1, "caos_05-10")    
save_images(period3, dates3, times3, 8, 1, "caos_10-15")    
save_images(period4, dates4, times4, 8, 1, "caos_15-20")    
save_images(period5, dates5, times5, 8, 1, "caos_20-24")    

In [16]:

years_period = ["2000", "2001", "2002", "2003", "2004", "2005"]

def get_swaths_for_labeling(years_period, labels_to_count=[i for i in range(14)], threshold=30, larger_than=True, labels_for_swath_size=[i for i in range(14)]):
    full_file_list = []
    dates = []
    times = []
    modis_start_date = 55
    for year in years_period:
        if year == years_period[0]:
            files_tmp = [f for f in os.listdir(modis_folder + year) if f.endswith("band_31.npz")]
            files_yr = []
            for f in files_tmp:
                if int(f.split(".")[1][5:]) >= modis_start_date:
                    files_yr.append(f) 
        elif year == years_period[-1]:
            files_tmp = [f for f in os.listdir(modis_folder + year) if f.endswith("band_31.npz")]
            files_yr = []
            for f in files_tmp:
                if int(f.split(".")[1][5:]) < modis_start_date:
                    files_yr.append(f) 
        else:
            files_yr = [f for f in os.listdir(modis_folder + year) if f.endswith("band_31.npz")]

        print([f.split(".")[1][1:] for f in files_yr])
        dates.append(([f.split(".")[1][1:] for f in files_yr]))
        times.append(([f.split(".")[2] for f in files_yr]))

        full_file_list.append(files_yr)
    full_file_list = np.concatenate(full_file_list)

    times_large_cao = []
    dates = np.concatenate(dates)
    times = np.concatenate(times)
    print(len(full_file_list))
    rand_idx = np.random.randint(0, len(full_file_list), 500)
    full_file_list = full_file_list[rand_idx]
    random_choice_dates = dates[rand_idx]
    random_choice_times = times[rand_idx]
    import glob

    date_list = []
    time_list = []
    x_list = []
    i= 0
    while len(x_list) < 50:
        print("try:", i)
        # Construct the file search pattern
        result, mask, lon_lat, valid_cols_lon  = process_npy_file(modis_folder + full_file_list[i].split(".")[1][1:5] + "/"+ full_file_list[i],
                    file_layers=None,
                    bands=[31],
                    max_zenith=47.4,
                    data_loc=data_loc,
                    full_water_mask=None,
                    tree=tree,
                    return_lon_lat=True,
                    npz_compressed=True,
                    lon_min=-45,
                    lon_max=65,
                    lat_min=55,
                    lat_max=82)
        mask = np.where(mask>0.1, 0, 1)
        print(result.shape)
        patches, all_lon_patches, all_lat_patches, starts, ends, shapes, n_patches_tot, indices = generate_patches([result],
                                                                                                                    [mask],
                                                                                                                    [lon_lat],
                                                                                                                    max_vals,
                                                                                                                    min_vals,
                                                                                                                    autoencoder_predict,
                                                                                                                    strides=[1, strides, strides,1])
        if len(patches) < 10:
            i+=1
            continue
        with tf.device('/CPU:0'):   
            encoded_patches_cao = encoder.predict(patches)
            encoded_patches_flat_cao = encoded_patches_cao.reshape(encoded_patches_cao.shape[0], -1)
        n_K = 14
        cluster = joblib.load(f"{data_loc}models/patch_size128/filter128/clustering/K{n_K}/cluster_{model_run_name}_filter128_K{n_K}.pkl" )
        labels = cluster.predict(encoded_patches_flat_cao)
        label_map, lon_map, lat_map = process_label_maps(labels,
                                                        all_lon_patches,
                                                        all_lat_patches,
                                                        starts,
                                                        ends,
                                                        shapes,
                                                        indices,
                                                        n_K,
                                                        n_patches_tot,
                                                        128,
                                                        strides,
                                                        [2, 10], 
                                                        [], 
                                                        size_thr_1=None, 
                                                        size_thr_2=None)
        binary_map = np.isin(label_map[0], labels_to_count)
        labeled_map, num_features = ndimage.label(binary_map)
        region_sizes = ndimage.sum(
            binary_map, labeled_map, range(num_features + 1))

        still_thershold = False
        # Loop through each region and check if the region size is below the thr
        # Skipping background (label 0)
        if larger_than:
            for region_label in range(1, num_features + 1):
                if region_sizes[region_label] >= threshold:
                    still_thershold = True
                    break
        else:
            binary_map_all = np.isin(label_map[0], labels_for_swath_size)
            labeled_map_swath_size, num_features_swath = ndimage.label(binary_map_all)
            region_sizes_all = ndimage.sum(
                binary_map_all, labeled_map_swath_size, range(num_features_swath + 1))
            
            for region_label in range(1, num_features + 1):
                if region_sizes[region_label] < threshold and np.any(region_sizes_all >= 30):
                    still_thershold = True
                    break
            
        if still_thershold:
            x_list.append(result)
            date_list.append(random_choice_dates[i])
            time_list.append(random_choice_times[i])

        i+=1
        print("current n images:", len(x_list))
    return x_list, date_list, time_list

In [336]:
years_period = ["2000", "2001", "2002", "2003", "2004", "2005"]
random1, random_dates1, random_times1 = get_swaths_for_labeling(years_period, labels_to_count=[i for i in range(14)], threshold=50, larger_than=True)
years_period = ["2005", "2006", "2007", "2008", "2009", "2010"]
random2, random_dates2, random_times2 = get_swaths_for_labeling(years_period, labels_to_count=[i for i in range(14)], threshold=50, larger_than=True)
years_period = ["2010", "2011", "2012", "2013", "2014", "2015"]
random3, random_dates3, random_times3 = get_swaths_for_labeling(years_period, labels_to_count=[i for i in range(14)], threshold=50, larger_than=True)
years_period = ["2015", "2016", "2017", "2018", "2019", "2020"]
random4, random_dates4, random_times4 = get_swaths_for_labeling(years_period, labels_to_count=[i for i in range(14)], threshold=50, larger_than=True)
years_period = ["2020", "2021", "2022", "2023", "2024"]
random5, random_dates5, random_times5 = get_swaths_for_labeling(years_period, labels_to_count=[i for i in range(14)], threshold=50, larger_than=True)

['2000362', '2000272', '2000139', '2000327', '2000346', '2000098', '2000106', '2000271', '2000350', '2000088', '2000065', '2000267', '2000073', '2000253', '2000329', '2000097', '2000083', '2000269', '2000361', '2000105', '2000079', '2000328', '2000353', '2000116', '2000351', '2000087', '2000139', '2000315', '2000249', '2000072', '2000117', '2000354', '2000360', '2000252', '2000327', '2000323', '2000298', '2000121', '2000058', '2000281', '2000132', '2000130', '2000278', '2000073', '2000092', '2000143', '2000067', '2000297', '2000072', '2000311', '2000280', '2000107', '2000109', '2000085', '2000285', '2000118', '2000337', '2000144', '2000086', '2000344', '2000327', '2000309', '2000074', '2000105', '2000089', '2000128', '2000349', '2000080', '2000278', '2000142', '2000134', '2000334', '2000115', '2000097', '2000122', '2000257', '2000080', '2000092', '2000328', '2000150', '2000244', '2000251', '2000074', '2000098', '2000322', '2000259', '2000266', '2000363', '2000286', '2000093', '2000284'

In [337]:
save_images(random1, random_dates1, random_times1, 8, 1, "random_00-05")    
save_images(random2, random_dates2, random_times2, 8, 1, "random_05-10")    
save_images(random3, random_dates3, random_times3, 8, 1, "random_10-15")    
save_images(random4, random_dates4, random_times4, 8, 1, "random_15-20")    
save_images(random5, random_dates5, random_times5, 8, 1, "random_20-24")    

In [17]:
years_period = ["2000", "2001", "2002", "2003", "2004", "2005"]
no_cao1, no_cao_dates1, no_cao_times1 = get_swaths_for_labeling(years_period, labels_to_count=[6, 9, 10, 2], threshold=3, larger_than=False, labels_for_swath_size=[i for i in range(14)])
years_period = ["2005", "2006", "2007", "2008", "2009", "2010"]
no_cao2, no_cao_dates2, no_cao_times2 = get_swaths_for_labeling(years_period, labels_to_count=[6, 9, 10, 2], threshold=3, larger_than=False, labels_for_swath_size=[i for i in range(14)])
years_period = ["2010", "2011", "2012", "2013", "2014", "2015"]
no_cao3, no_cao_dates3, no_cao_times3 = get_swaths_for_labeling(years_period, labels_to_count=[6, 9, 10, 2], threshold=3, larger_than=False, labels_for_swath_size=[i for i in range(14)])
years_period = ["2015", "2016", "2017", "2018", "2019", "2020"]
no_cao4, no_cao_dates4, no_cao_times4 = get_swaths_for_labeling(years_period, labels_to_count=[6, 9, 10, 2], threshold=3, larger_than=False, labels_for_swath_size=[i for i in range(14)])
years_period = ["2020", "2021", "2022", "2023", "2024"]
no_cao5, no_cao_dates5, no_cao_times5 = get_swaths_for_labeling(years_period, labels_to_count=[6, 9, 10, 2], threshold=3, larger_than=False, labels_for_swath_size=[i for i in range(14)])

['2000362', '2000272', '2000139', '2000327', '2000346', '2000098', '2000106', '2000271', '2000350', '2000088', '2000065', '2000267', '2000073', '2000253', '2000329', '2000097', '2000083', '2000269', '2000361', '2000105', '2000079', '2000328', '2000353', '2000116', '2000351', '2000087', '2000139', '2000315', '2000249', '2000072', '2000117', '2000354', '2000360', '2000252', '2000327', '2000323', '2000298', '2000121', '2000058', '2000281', '2000132', '2000130', '2000278', '2000073', '2000092', '2000143', '2000067', '2000297', '2000072', '2000311', '2000280', '2000107', '2000109', '2000085', '2000285', '2000118', '2000337', '2000144', '2000086', '2000344', '2000327', '2000309', '2000074', '2000105', '2000089', '2000128', '2000349', '2000080', '2000278', '2000142', '2000134', '2000334', '2000115', '2000097', '2000122', '2000257', '2000080', '2000092', '2000328', '2000150', '2000244', '2000251', '2000074', '2000098', '2000322', '2000259', '2000266', '2000363', '2000286', '2000093', '2000284'

In [18]:
save_images(no_cao1, no_cao_dates1, no_cao_times1, 8, 1, "nocaos_00-05")    
save_images(no_cao2, no_cao_dates2, no_cao_times2, 8, 1, "nocaos_05-10")    
save_images(no_cao3, no_cao_dates3, no_cao_times3, 8, 1, "nocaos_10-15")    
save_images(no_cao4, no_cao_dates4, no_cao_times4, 8, 1, "nocaos_15-20")    
save_images(no_cao5, no_cao_dates5, no_cao_times5, 8, 1, "nocaos_20-24")    

In [56]:
files = os.listdir("/uio/kant/geo-geofag-u1/fslippe/data/figures_for_labeling/full_modis_record/dataset_500/")
files = [f for f in files if f.endswith(".jpg")]
existing_files = ["MOD021KM.A2007312.1135", "MOD021KM.A2020022.1350", "MOD021KM.A2010055.2125"]
for file in files:
    if file.split("_")[0] in existing_files:
        print(file)
    # existing_files.append(file.split("_")[0])

MOD021KM.A2020022.1350_caos_20-24.jpg
MOD021KM.A2007312.1135_random_05-10.jpg
MOD021KM.A2007312.1135_caos_05-10.jpg
MOD021KM.A2020022.1350_caos_15-20.jpg
MOD021KM.A2010055.2125_caos_10-15.jpg
MOD021KM.A2010055.2125_caos_05-10.jpg


In [43]:
files = os.listdir("/uio/kant/geo-geofag-u1/fslippe/data/figures_for_labeling/full_modis_record/dataset_500/")
files = [f.split("_")[0] for f in files if f.endswith(".jpg")]
npz_folder = "/mn/vann/fslippe/modis/MOD02_npz/"
target_directory = "/mn/vann/fslippe/modis/labeling_data/full_record_500/"
import shutil
for f in files:
    year = f.split(".")[1][1:5]
    folder_year = os.path.join(npz_folder, f"{year}/")
    
    # Create the pattern with the wildcard
    pattern = os.path.join(folder_year, f"{f}*.npz")
    
    # Use glob to find the matching file(s)
    matching_files = glob.glob(pattern)
    result, mask, lon_lat, valid_cols_lon  = process_npy_file(matching_files[0],
                    file_layers=None,
                    bands=[31],
                    max_zenith=47.4,
                    data_loc=data_loc,
                    full_water_mask=None,
                    tree=tree,
                    return_lon_lat=True,
                    npz_compressed=True,
                    lon_min=-45,
                    lon_max=65,
                    lat_min=55,
                    lat_max=82)
    filename = matching_files[0].split("/")[-1]
    np.savez_compressed(f"{target_directory}{filename}", data=result, mask=mask, lon=lon_lat[0,:,:], lat=lon_lat[1,:,:])
    
    # for matching_file in matching_files:
    #     # Define the destination path
    #     destination = os.path.join(target_directory, os.path.basename(matching_file))
        
    #     # Copy the file to the target directory
    #     shutil.copy(matching_file, destination)

    #     # If you need confirmation
    #     print(f"Copied {matching_file} to {destination}")

In [ ]:
all_files = os.listdir("full_modis_record/")
cao_files = [f for f in all_files if "_caos_" in f]
nocao_files = [f for f in all_files if "_nocaos_" in f]
random_files = [f for f in all_files if "_random_" in f]

ends = ["00-05.jpg", "05-10.jpg", "10-15.jpg", "15-20.jpg", "20-24.jpg"]
files_final = []
files_rest = []

for files in [cao_files, nocao_files, random_files]:
    for end in ends:
        rand_idx = np.random.choice(50, 25, replace=False)
        rest_idx = np.setdiff1d(np.arange(50), rand_idx)

        files_w_end = np.array([f for f in files if f.endswith(end)])
        files_final.append(files_w_end[rand_idx])
        files_rest.append(files_w_end[rest_idx])
      
files_rest = np.concatenate(files_rest)
files_final = np.concatenate(files_final)

rand_idx = np.random.choice(375, 125, replace=False)
files_all_random = files_rest[rand_idx]

In [ ]:
import shutil
files_full = np.concatenate([files_all_random, files_final])


source_dir = "prepared data"
destination_dir = "final labeling dataset/"
for file in files_full:
    source_file = f"{source_dir}{file}"
    destination_file = f"{destination_dir}{file}"
    shutil.copy(source_file, destination_file)


In [ ]:
full_file_list = []
years_period = ["2000", "2001", "2002", "2003", "2004", "2005"]
modis_start_date = 55
for year in years_period:
    if year == years_period[0]:
        files_tmp = os.listdir(modis_folder + year)
        files_yr = []
        for f in files_tmp:
            if int(f.split(".")[1][5:]) >= modis_start_date:
                files_yr.append(f) 
    elif year == years_period[-1]:
        files_tmp = os.listdir(modis_folder + year)
        files_yr = []
        for f in files_tmp:
            if int(f.split(".")[1][5:]) < modis_start_date:
                files_yr.append(f) 
    else:
        files_yr = os.listdir(modis_folder + year)
    
    full_file_list.append(files_yr)
full_file_list = np.concatenate(full_file_list)
